In [2]:
import pandas as pd

In [114]:
work_with_prefix = "big"

In [117]:
df = pd.read_csv("axel_data/axel_pressedaten_{}.csv".format(work_with_prefix))#, delimiter='\t')

print("Loaded {} examples".format(df.shape[0]))
print(df.columns)
new_cols = []
for i in df.columns:
    if 'Unnamed' not in i and i != "cmsid_info":
        new_cols.append(i)
df = df[new_cols]
print(df.columns)

df.head()

Loaded 506794 examples
Index(['Unnamed: 0', 'cmsid', 'title', 'fb_likes', 'description', 'content',
       'fb_shares', 'fb_likes.1', 'fb_comments', 'fb_total', 'fb_total2',
       'tw_count', 'total', 'total2', 'total3', 'cmsid_info'],
      dtype='object')
Index(['cmsid', 'title', 'fb_likes', 'description', 'content', 'fb_shares',
       'fb_likes.1', 'fb_comments', 'fb_total', 'fb_total2', 'tw_count',
       'total', 'total2', 'total3'],
      dtype='object')


/Users/MA573RWARR10R/anaconda/envs/pwsankaf/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cmsid,title,fb_likes,description,content,fb_shares,fb_likes.1,fb_comments,fb_total,fb_total2,tw_count,total,total2,total3
0,44184246,Kadaver zerlegt - Fischernetz im Pottwal-Magen...,7,Die toten Pottwale aus der Nordsee werden zerl...,Nordstrand ` Verblüffende Entdec...,8.0,7.0,6.0,21.0,15.0,0.0,15.0,23.0,15.0
1,44166990,25. Thronjubiläum in Norwegen: Harald &amp; So...,1,Norwegen feiert am 17. Januar das silberne Thr...,Heute vor 25 Jahren wurden aus Ha...,5.0,1.0,0.0,6.0,6.0,0.0,6.0,11.0,6.0
2,44144016,Vaporcade Jupiter wird zur Kippe am Ohr: Mit d...,156,Smartphones sind bekanntlich Multi-Talente. Si...,Smartphones sind ja bekanntlich M...,126.0,156.0,322.0,604.0,282.0,0.0,282.0,408.0,282.0
3,44170768,"Johannes Kühn verballert fast alles, Deutschla...",0,Biathlet Johannes K?hn leistete sich beim Welt...,Gar nicht kühn von Johannes Kühn ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,44189394,Comeback bei Union Berlin: Benjamin Köhler nac...,1689,Das ist Gl?ck pur! Benjamin K?hler (35) hat de...,Das ist Glück pur! Benjamin Köhle...,29.0,1689.0,23.0,1741.0,1718.0,0.0,1718.0,1747.0,1718.0


In [119]:
preprocess = False

if preprocess:
    N = None
    
    if N is not None:
        df_small = df[:N]
        
    df_small['cmsid_info'] = df['cmsid'].apply(lambda x: x.isdigit())
    del df

    df = df_small
    df = df[df['cmsid_info']==True]
    df['cmsid'] = df['cmsid'].apply(lambda x: int(x))
    df.drop(['cmsid_info'], axis=1)
    
df.set_index('cmsid', inplace=True)

In [120]:
df.shape

(506794, 13)

# make is_viral

In [121]:
df_descr = df.describe()
quartile_75 = df_descr['total']['75%']

df['is_viral'] = df['total'] > quartile_75
df['is_viral'] = df.is_viral.apply(lambda x:int(x))

df.head()

,title,fb_likes,description,content,fb_shares,fb_likes.1,fb_comments,fb_total,fb_total2,tw_count,total,total2,total3,is_viral
cmsid,,,,,,,,,,,,,,
44184246,Kadaver zerlegt - Fischernetz im Pottwal-Magen...,7,Die toten Pottwale aus der Nordsee werden zerl...,Nordstrand ` Verblüffende Entdec...,8.0,7.0,6.0,21.0,15.0,0.0,15.0,23.0,15.0,0
44166990,25. Thronjubiläum in Norwegen: Harald &amp; So...,1,Norwegen feiert am 17. Januar das silberne Thr...,Heute vor 25 Jahren wurden aus Ha...,5.0,1.0,0.0,6.0,6.0,0.0,6.0,11.0,6.0,0
44144016,Vaporcade Jupiter wird zur Kippe am Ohr: Mit d...,156,Smartphones sind bekanntlich Multi-Talente. Si...,Smartphones sind ja bekanntlich M...,126.0,156.0,322.0,604.0,282.0,0.0,282.0,408.0,282.0,1
44170768,"Johannes Kühn verballert fast alles, Deutschla...",0,Biathlet Johannes K?hn leistete sich beim Welt...,Gar nicht kühn von Johannes Kühn ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
44189394,Comeback bei Union Berlin: Benjamin Köhler nac...,1689,Das ist Gl?ck pur! Benjamin K?hler (35) hat de...,Das ist Glück pur! Benjamin Köhle...,29.0,1689.0,23.0,1741.0,1718.0,0.0,1718.0,1747.0,1718.0,1


In [124]:
try:
    df.rename({'description':'lead', 'fb_likes.1':'fb_likes_1'}, axis=1, inplace=True)
except Exception as e:
    pass

useful_cols = ['title', 'lead', 'content', 'total', 'is_viral']

df = df[useful_cols]
df.head()

,title,lead,content,total,is_viral
cmsid,,,,,
44184246,Kadaver zerlegt - Fischernetz im Pottwal-Magen...,Die toten Pottwale aus der Nordsee werden zerl...,Nordstrand ` Verblüffende Entdec...,15.0,0
44166990,25. Thronjubiläum in Norwegen: Harald &amp; So...,Norwegen feiert am 17. Januar das silberne Thr...,Heute vor 25 Jahren wurden aus Ha...,6.0,0
44144016,Vaporcade Jupiter wird zur Kippe am Ohr: Mit d...,Smartphones sind bekanntlich Multi-Talente. Si...,Smartphones sind ja bekanntlich M...,282.0,1
44170768,"Johannes Kühn verballert fast alles, Deutschla...",Biathlet Johannes K?hn leistete sich beim Welt...,Gar nicht kühn von Johannes Kühn ...,0.0,0
44189394,Comeback bei Union Berlin: Benjamin Köhler nac...,Das ist Gl?ck pur! Benjamin K?hler (35) hat de...,Das ist Glück pur! Benjamin Köhle...,1718.0,1


In [125]:
df.to_csv("axel_data/axel_pressedaten_{}_cleared.csv".format(work_with_prefix))